In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains, ScrollOrigin
import pandas as pd
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
url = 'https://www.thegioididong.com/muc-in#c=1262&o=7&pi=0'

driver = webdriver.Chrome()
driver.get(url)

### Lấy danh sách sản phẩm

In [27]:
listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')
len(product_list)

131

#### Tạo Func lấy thông 1 tin sản phẩm

In [28]:
def get_product_info(product_elm):
    phone_name = product_elm.find_element(By.XPATH, './/h3').text

    try:
        phone_price = product_elm.find_element(By.XPATH, './/strong[@class="price"]').text
    except:
        phone_price = "N/A"

    
    if phone_price == "N/A":
        try:
            phone_price = product_elm.find_element(By.CLASS_NAME, 'box-price-present').text
        except:
            phone_price = "N/A"
        
    result = {
        'name': phone_name,
        'price': phone_price
    }
    return result

In [6]:
# Hàm lấy review của tất cả điện thoại
all_reviews = []
tmp = 0
def getReviewAllPhone(product_list):
    for item in product_list:
        product_info = get_product_info(item)
        item.click()
        sleep(2)

        try:
            # Nếu mà có readmore thì mình bấm luôn
            # read_more_elm = driver.find_element(By.XPATH, '/html/body/section/div[2]/div[1]/div[11]/div[2]/div/div/div[6]/div/a')
            read_more_elm = driver.find_element(By.CLASS_NAME, 'btn-view-all')
            if read_more_elm:
                read_more_elm.click()
                sleep(2)

                while True:
                    reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                    for review in reviews:
                        try:
                            all_reviews.append({
                                'product_name': product_info['name'],
                                'product_price': product_info['price'],
                                'review': review.text
                            })
                        except:
                            continue

                    try:
                        page_elm = driver.find_element(By.CLASS_NAME, 'pagcomment')
                        list_page = page_elm.find_elements(By.XPATH, './/a')

                        next_page = None # Init var
                        for page in list_page:
                            if page.text.strip() == "»" or page.text.strip() == "›":
                                next_page = page
                                break

                        if next_page: # if next_page exist
                            next_page.click() # forward
                            sleep(2)
                        else:
                            last_page = list_page[-1]
                            last_page.click()
                            sleep(2)
                            # Lấy xong hết review của trang cuối cùng của sản phẩm
                            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                            for review in reviews:
                                try:
                                    all_reviews.append({
                                        'product_name': product_info['name'],
                                        'product_price': product_info['price'],
                                        'review': review.text
                                    })
                                except:
                                    continue
                            driver.back()
                            driver.back()
                            sleep(2)
                            break
                    except:
                        print("Không còn trang tiếp theo!")
                        driver.back()
                        driver.back()
                        sleep(2)
                        break
        except:

            # Nếu có review nhưng chỉ có ===>> 1 TRANG
            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
            for review in reviews:
                all_reviews.append({
                    'product_name': product_info['name'],
                    'product_price': product_info['price'],
                    'review': review.text
                })
            
            # Lấy xong thì back về
            driver.back()
            sleep(2)
            listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
            product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'ajaxed')

In [29]:
all_reviews = []
tmp = 0

def getReviewAllPhone_v02(product_list):
    global tmp
    i = 0  # Bắt đầu từ 64
    while i < len(product_list):
        # Lấy thông tin sản phẩm và click vào sản phẩm
        product_info = get_product_info(product_list[i])
        item = product_list[i]
        item.click()
        sleep(2)

        try:
            # Nếu có nút "Xem thêm" thì bấm
            read_more_elm = driver.find_element(By.CLASS_NAME, 'btn-view-all')
            if read_more_elm:
                read_more_elm.click()
                sleep(2)

                while True:
                    reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                    for review in reviews:
                        try:
                            all_reviews.append({
                                'product_name': product_info['name'],
                                'product_price': product_info['price'],
                                'review': review.text
                            })
                        except:
                            continue

                    try:
                        page_elm = driver.find_element(By.CLASS_NAME, 'pagcomment')
                        list_page = page_elm.find_elements(By.XPATH, './/a')

                        next_page = None
                        for page in list_page:
                            if page.text.strip() == "»" or page.text.strip() == "›":
                                next_page = page
                                break

                        if next_page:
                            next_page.click()
                            sleep(2)
                        else:
                            last_page = list_page[-1]
                            last_page.click()
                            sleep(2)
                            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                            for review in reviews:
                                try:
                                    all_reviews.append({
                                        'product_name': product_info['name'],
                                        'product_price': product_info['price'],
                                        'review': review.text
                                    })
                                except:
                                    continue
                            driver.back()
                            driver.back()
                            sleep(2)
                            break
                    except:
                        print("Không còn trang tiếp theo!")
                        driver.back()
                        driver.back()
                        sleep(2)
                        break
        except:
            # Nếu chỉ có 1 trang review
            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
            for review in reviews:
                all_reviews.append({
                    'product_name': product_info['name'],
                    'product_price': product_info['price'],
                    'review': review.text
                })
            
            driver.back()
            sleep(2)

        # Sau khi quay lại, làm mới product_list
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'listproduct'))
        )
        listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
        product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')
        print(f"Độ dài product_list sau khi làm mới: {len(product_list)}")
        
        i += 1  # Tăng i để xử lý sản phẩm tiếp theo

In [30]:
getReviewAllPhone_v02(product_list)

Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 131
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 131
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 131
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 131
Độ dài product_list sau khi làm mới: 131
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 131
Độ dài product_list sau khi làm mới: 131
Độ dài product_list sau khi làm mới: 131
Độ dài product_list sau khi làm mới: 131
Độ dài product_list sau khi làm mới: 131
Độ dài product_list sau khi làm mới: 131
Độ dài product_list sau khi làm mới: 131
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 131
Độ dài product_list sau khi làm mới: 131
Độ dài product_list sau khi làm mới: 131
Độ dài product_list sau khi làm mới: 131
Độ dài product_list sau khi làm mới: 131
Độ dài product_list sau khi làm mới: 131
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 131
Độ dài product_list sau khi làm 

In [31]:
all_reviews

[{'product_name': 'Tai nghe TWS Samsung Galaxy Buds3 Pro R630N',
  'product_price': '5.090.000₫',
  'review': 'Pin tại nghe sài được 4 tiếng để 7 tiếng. Thất vọng.'},
 {'product_name': 'Tai nghe TWS Samsung Galaxy Buds3 Pro R630N',
  'product_price': '5.090.000₫',
  'review': 'Sạc lúc được lúc ko thua xa buds 2'},
 {'product_name': 'Tai nghe TWS Samsung Galaxy Buds3 Pro R630N',
  'product_price': '5.090.000₫',
  'review': 'Sạc không vào, mau hết pin thời lượng pin thực tế chỉ từ 3h đến 5h dù không mở chống ồn'},
 {'product_name': 'Tai nghe TWS Samsung Galaxy Buds3 Pro R630N',
  'product_price': '5.090.000₫',
  'review': 'Tai nghe buds3 pro chất âm nghe rất hay , khử tiếng ồn tốt không có gì để chê , nhưng mà nghe lâu sẽ đau tai , các bạn tai nhỏ như mình nên cân nhắc nhé .'},
 {'product_name': 'Tai nghe TWS Samsung Galaxy Buds3 Pro R630N',
  'product_price': '5.090.000₫',
  'review': 'Chiếc tai nghe này quả thực rất thu hút với thiết kế đẹp mắt và chất âm tuyệt vời. Khả năng chống ồn c

In [32]:
df = pd.DataFrame(all_reviews)
# Lưu file
df.to_csv("E:\\study\\tu_hoc\\DE\\CRAWL_DATA\\review_data\\data\\head_phone_bluetooth_data.xlsx", index=False)